In [ ]:
import pandas as pd
from prophet import Prophet
# Load the data
file_path = '../../Data/Scripts/dataProcesing/TimeSort.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Combine Date and Time Period into a single Datetime column if not already done
# data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time Period'])

# Check for NaN values in 'Datetime' column
if data['Datetime'].isnull().any():
    # Option 1: Remove rows with NaN values
    # data = data.dropna(subset=['Datetime'])

    # Option 2: Impute missing values (forward fill, backward fill, or interpolation)
    data['Datetime'].fillna(method='ffill', inplace=True)

    # Add your chosen method of handling NaN values here

# Prepare data for Prophet
prophet_data = data[['Datetime', 'Min. per HM']].rename(columns={'Datetime': 'ds', 'Min. per HM': 'y'})

# Initialize and fit the Prophet model
model = Prophet()
model.fit(prophet_data)

# Create future dataframe for predictions
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)

# (Optional) Plot the forecast
# forecast.plot()

